In [ ]:
import pandas as pd
# Importing pandas library for data manipulation and analysis
import numpy as np
import matplotlib.pyplot as plt
# Importing matplotlib for data visualization
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/superstore.csv')
# Reading the CSV file into a pandas DataFrame

In [ ]:
df.head()
# Displaying the first few rows of the DataFrame

#Data Understanding 

In [ ]:
data=df.copy()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()
# Displaying a summary of the DataFrame including data types and non-null values

In [ ]:
df.describe()
# Displaying descriptive statistics of numerical columns

In [ ]:
columns_to_display = [
    'Order.Priority', 'Product.ID', 'Product.Name', 'Profit',
    'Quantity', 'Region', 'Row.ID'
]

df_selected = df[columns_to_display]
df_selected.head()
# Displaying the first few rows of the DataFrame

In [ ]:
df['Row.ID'].max()

In [ ]:
df['Category'].value_counts()
# Counting unique values in a column

In [ ]:
df.groupby('Category')['Profit'].sum().sort_values(ascending=False)    #EDA
# Grouping data and calculating the sum for each group

#Data Cleaning 

In [ ]:
df.isna().sum()   #view missing value 

In [ ]:
df.drop('Customer.Name',inplace=True,axis=1)

In [ ]:
df.drop('记录数',inplace=True,axis=1)

In [ ]:
df.drop('Market',inplace=True,axis=1)

In [ ]:
df['Customer.ID'] = df['Customer.ID'].str.replace(r'\D', '', regex=True)      #remove char

In [ ]:
df['Customer.ID'] = df['Customer.ID'].astype(int)   #change it to int 

In [ ]:
df['Order.Date'] = pd.to_datetime(df['Order.Date'])   #make it numeric 

In [ ]:
df['Month_Order'] = df['Order.Date'].dt.month.astype(str).str.zfill(2)
df['Day_Order'] = df['Order.Date'].dt.day.astype(str).str.zfill(2)               #make new columns (month,day )

In [ ]:
df.head()
# Displaying the first few rows of the DataFrame

In [ ]:
df['Ship.Date'] = pd.to_datetime(df['Ship.Date'])   #make it numeric 

In [ ]:
df.head()
# Displaying the first few rows of the DataFrame

#EDA

In [ ]:
# This code ranks product categories based on their average profit, from highest to lowest
df.groupby('Category')['Profit'].mean().sort_values(ascending=False)

In [ ]:
df['Category'].value_counts()
# Counting unique values in a column

In [ ]:
# Step 1: Summarize total profit and quantity sold for each region
region_summary = df.groupby('Region').agg(
    Total_Profit=('Profit', 'sum'),         # Sum of profits per region
    Total_Quantity=('Quantity', 'sum')      # Sum of quantities sold per region
).reset_index()

# Step 2: Find the most frequent category in each region
top_category = (
    df.groupby(['Region', 'Category'])
    .size()                                  # Count how many times each category appears per region
    .reset_index(name='Count')               # Convert the group count to a DataFrame
    .sort_values(['Region', 'Count'], ascending=[True, False])  # Sort within each region by frequency
    .drop_duplicates(subset='Region')        # Keep only the most frequent category per region
    .rename(columns={'Category': 'Top_Category'})  # Rename the column for clarity
    [['Region', 'Top_Category']]             # Select only needed columns
)

# Step 3: Merge the summary with the top category info
final_result = pd.merge(region_summary, top_category, on='Region')

# Step 4: Sort the result by total profit in descending order
final_result = final_result.sort_values('Total_Profit', ascending=False).reset_index(drop=True)

# Step 5: Display the final result
print(final_result)


In [ ]:
ship_Classes=df['Ship.Mode'].value_counts()
# Counting unique values in a column

In [ ]:
# Plotting a pie chart to show the distribution of shipping classes
plt.pie(
    ship_Classes,                      # Data values for each shipping class
    labels=ship_Classes.index,        # Labels taken from the index (shipping class names)
    startangle=90,                    # Rotate the start of the pie chart for better appearance
    autopct='%1.1f%%'                 # Show percentage on each slice with 1 decimal
)
plt.title('Ship Classes Distribution')  # Title of the chart
plt.show()                              # Display the pie chart


In [ ]:
# Calculating the average sales for each year and sorting them from highest to lowest
df.groupby('Year')['Sales'].mean().sort_values(ascending=False)


In [ ]:
df.groupby('Year')['Profit'].sum().sort_values(ascending=False)
# Grouping data and calculating the sum for each group

In [ ]:
# Counting how many times each product category appears in each year,
# and sorting the counts within each year in descending order
df.groupby('Year')['Category'].value_counts(ascending=False)


In [ ]:
# Step 1: Calculate total profit for each (Year, Category) combination and sort
yearly_profit = (
    df.groupby(['Year', 'Category'])['Profit']
    .sum()  # Sum of profits per category per year
    .reset_index()
    .sort_values(['Year', 'Profit'], ascending=[True, False])  # Sort by year and then by profit descending
)

# Step 2: Print results in a formatted way, separating each year with a line
prev_year = None
for _, row in yearly_profit.iterrows():
    year = row['Year']
    category = row['Category']
    profit = row['Profit']

    # Print a separator when moving to a new year
    if year != prev_year and prev_year is not None:
        print('##########')

    # Print year, category, and profit
    print(f"{year} | {category} | Profit: {profit:.2f}")
    prev_year = year


In [ ]:
# Creating a pivot table that counts the number of orders for each year and discount value.
# Missing combinations are filled with 0. Columns (discount values) are sorted in descending order.
df.pivot_table(
    index='Year',                      # Rows represent each year
    columns='Discount',               # Columns represent each unique discount value
    aggfunc='size',                   # Count the number of records for each combination
    fill_value=0                      # Replace NaNs with 0 for missing combinations
).sort_index(axis=1, ascending=False) # Sort the discount columns from high to low


In [ ]:
# Calculating the average profit for each discount value,
# and sorting the results by discount value in descending order
df.groupby('Discount')['Profit'].mean().sort_index(ascending=False)

In [ ]:
df['Discount'].value_counts().sort_index(ascending=False)
# Counting unique values in a column

In [ ]:
df.columns

In [ ]:
# Convert the 'Order.Date' column to datetime format
df['Order.Date'] = pd.to_datetime(df['Order.Date'])

# Define the snapshot date as one day after the latest order date
# This is often used as a reference point for customer analysis (e.g., RFM)
snapshot_date = df['Order.Date'].max() + pd.Timedelta(days=1)


In [ ]:
# Step 1: Create RFM metrics for each customer
rfm = df.groupby('Customer.ID').agg({
    'Order.Date': lambda x: (snapshot_date - x.max()).days,  # Recency: Days since last purchase
    'Order.ID': 'nunique',                                   # Frequency: Number of unique orders
    'Profit': 'sum'                                          # Monetary: Total profit from the customer
}).reset_index()

# Step 2: Rename the columns to RFM standard names
rfm.rename(columns={
    'Order.Date': 'Recency',
    'Order.ID': 'Frequency',
    'Profit': 'Monetary'
}, inplace=True)


In [ ]:
print(rfm.columns)

In [ ]:
# Assign RFM scores from 1 to 5 based on quantiles

# Recency score: Lower recency (more recent purchases) gets higher score (5 = best)
df['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Frequency score: Higher frequency gets higher score (5 = best)
# .rank() used to handle ties in frequency values before binning
df['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Monetary score: Higher monetary value gets higher score (5 = best)
df['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])


In [ ]:
df.head()
# Displaying the first few rows of the DataFrame

In [ ]:
df.groupby('Country')['Customer.ID'].count().sort_values()    #new one

In [ ]:
#Year-over-Year Growth Rat
profit_2011 = df[df['Year'] == 2011]['Profit'].sum()
cost_2011 = df[df['Year'] == 2011]['Shipping.Cost'].sum()
ratio_2011 = profit_2011 / cost_2011

profit_2012 = df[df['Year'] == 2012]['Profit'].sum()
cost_2012 = df[df['Year'] == 2012]['Shipping.Cost'].sum()
ratio_2012 = profit_2012 / cost_2012

profit_2013 = df[df['Year'] == 2013]['Profit'].sum()
cost_2013 = df[df['Year'] == 2013]['Shipping.Cost'].sum()
ratio_2013 = profit_2013 / cost_2013

profit_2014 = df[df['Year'] == 2014]['Profit'].sum()
cost_2014 = df[df['Year'] == 2014]['Shipping.Cost'].sum()
ratio_2014 = profit_2014 / cost_2014

In [ ]:
print(f"2011: {ratio_2011:.2f}")
print(f"2012: {ratio_2012:.2f}")
print(f"2013: {ratio_2013:.2f}")
print(f"2014: {ratio_2014:.2f}")

In [ ]:
# Converting yearly growth ratios into percentage change from the previous year

percentage_2011 = (ratio_2011 - 1) * 100  # Percentage change in 2011
percentage_2012 = (ratio_2012 - 1) * 100  # Percentage change in 2012
percentage_2013 = (ratio_2013 - 1) * 100  # Percentage change in 2013
percentage_2014 = (ratio_2014 - 1) * 100  # Percentage change in 2014

In [ ]:
print(f"2011: {percentage_2011:.2f}%")
print(f"2012: {percentage_2012:.2f}%")
print(f"2013: {percentage_2013:.2f}%")
print(f"2014: {percentage_2014:.2f}%")

In [ ]:
# Counting the number of orders per year by counting non-null entries in the 'Country' column
df.groupby('Year')['Country'].count()


In [ ]:
# Counting the number of unique countries that placed orders in each year
unique_countries_per_year = df.groupby('Year')['Country'].nunique()

# Display the result
unique_countries_per_year


#Feature Engineering 

In [ ]:
df.columns

In [ ]:
# Extract the weekday name from the order date (e.g., Monday, Tuesday, etc.)
df['Order Weekday'] = df['Order.Date'].dt.day_name()

# Calculate the delivery duration in days between the shipping date and the order date
df['Delivery Days'] = (df['Ship.Date'] - df['Order.Date']).dt.days


In [ ]:
# Calculate the profit made per individual item sold
df['Profit per Item'] = df['Profit'] / df['Quantity']

In [ ]:
# Calculate the price per individual item sold (unit price)
df['Unit Price'] = df['Sales'] / df['Quantity']

In [ ]:
# Apply one-hot encoding to the 'Segment', 'Category', and 'Ship.Mode' columns
# drop_first=True is used to avoid multicollinearity (dummy variable trap)
df = pd.get_dummies(df, columns=['Segment', 'Category', 'Ship.Mode'], drop_first=True)

In [ ]:
# Encode the 'Country' column by replacing each country with its average profit
# This is a form of target (mean) encoding
df['Country_encoded'] = df.groupby('Country')['Profit'].transform('mean')

In [ ]:
# Encode the 'Product.Name' column by replacing each product with its average profit
# This helps represent product performance numerically
df['product_encoded'] = df.groupby('Product.Name')['Profit'].transform('mean')

In [ ]:
df.head()
# Displaying the first few rows of the DataFrame

In [ ]:
# Grouping the data by 'Region' and calculating the total profit for each region
df_region_profit = df.groupby('Region')['Profit'].sum().reset_index()

# Sorting the regions by total profit in descending order
df_region_profit = df_region_profit.sort_values(by='Profit', ascending=False)

In [ ]:
import plotly.express as px

# Create a bar chart showing total profit by region
fig = px.bar(
    df_region_profit,
    x='Region',
    y='Profit',
    title='Total Profit by Region',  # Chart title
    labels={'Profit': 'Profit', 'Region': 'Region'},  # Axis labels
    color='Profit',  # Color bars based on profit value
    color_continuous_scale='Teal'  # Color scale
)

# Rotate x-axis labels for better readability
fig.update_layout(xaxis_tickangle=-30)

# Display the chart
fig.show()
